In [3]:
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
import json
import sqlite3

In [4]:
connection = sqlite3.connect("camera_violations.db")
c = connection.cursor()

s = ''' select ADDRESS, CAMERAID, sum(VIOLATIONS) AS 'TOTAL_VIOLATIONS', LATITUDE, LONGITUDE from speed group by CAMERAID '''
c.execute(s)
table = c.fetchall()

df = pd.DataFrame(table)
df = df.rename(columns = {0:'address',1:'camera_id',2:'violations',3:'lat',4:'lon'})

s2 = ''' SELECT INTERSECTION, CAMERAID, ADDRESS, SUM(VIOLATIONS) as 'TOTAL_VIOLATIONS', LATITUDE, LONGITUDE FROM red_light group by ADDRESS '''
c.execute(s2)
table2 = c.fetchall()

df2 = pd.DataFrame(table2)
df2 = df2.rename(columns = {0:'intersection',1:'camera_id', 2:'address',3:'violations',4:'lat',5:'lon'})

In [5]:
sizeref_s = 2. * df['violations'].max() / (8 ** 2)
sizeref_r = 2. * df2['violations'].max() / (8 ** 2)

In [7]:
f2 = open("chicago.geojson",)
zips = json.load(f2)

In [8]:
fig = go.Figure(go.Choroplethmapbox(geojson=zips, locations=(df['lat'],df['lon']))) #, \
                                #z=zhvi_county_inc_pop["2021_2yr_increase"],
                                #colorscale="Inferno_r", zmin=-17, zmax=58, text=zhvi_county_inc_pop["text_2yrs"], 
                                #marker_opacity=opac, marker_line_width=0))
fig.update_layout(mapbox_style="carto-positron",
                mapbox_zoom=10, mapbox_center = {"lat": 41.9, "lon": -87.6})

fig.add_trace(go.Scattermapbox(
            lon = df['lon'],
            lat = df['lat'],
            mode = 'markers',
            marker = dict(
                size = df['violations']/sizeref_s,
                color = 'blue'
                    )))
fig.add_trace(go.Scattermapbox(
            lon = df2['lon'],
            lat = df2['lat'],
            mode = 'markers',
            marker = dict(
                size = df2['violations']/sizeref_r,
                color = 'red'
                    )))
       
        

fig.show()